In [1]:
import os
os.chdir("../")

In [2]:
import glob

mscx_path_list = []
for item in glob.glob("./xmls/mscx/*"):
    mscx_path_list.append(os.path.abspath(item))
mscx_path_list.sort()

tstree_path_list = []
for item in glob.glob("./xmls/tstree/*"):
    tstree_path_list.append(os.path.abspath(item))
tstree_path_list.sort()

In [3]:
from alm.comparator import tree_similarity_calculator as TSCalculator

tree_similarities = TSCalculator.calc_tree_similarities(mscx_path_list, tstree_path_list)

for item in tree_similarities:
    print(item.calc_similarity(), item.song_name)

0.2222222222222222 BE_FREE_A
0.16981132075471697 BE_FREE_S
0.15151515151515152 SUN_SHINE!!!_A
0.13793103448275862 SUN_SHINE!!!_S
0.21568627450980393 beautiful_days_A
0.13333333333333333 beautiful_days_S
0.1487603305785124 no_more_war_A
0.09090909090909091 no_more_war_S
0.19607843137254902 たけてん_A
0.12987012987012986 たけてん_S
0.08 オレンジ_A
0.2631578947368421 オレンジ_S
0.14814814814814814 キセキ_A
0.16049382716049382 キセキ_S
0.11607142857142858 サヨナラから始めよう_A
0.05555555555555555 サヨナラから始めよう_S
0.2972972972972973 ビリーヴ_A
0.1951219512195122 ビリーヴ_S
0.18095238095238095 ルーキーズ_A
0.15841584158415842 ルーキーズ_S
0.13513513513513514 人_A
0.11688311688311688 人_S
0.06382978723404255 刹那_A
0.08333333333333333 刹那_S
0.1651376146788991 君想い_A
0.1267605633802817 君想い_S
0.16216216216216217 愛唄_A
0.22807017543859648 愛唄_S
0.20689655172413793 扉_A
0.18518518518518517 扉_S
0.21052631578947367 旅立ち_A
0.1724137931034483 旅立ち_S
0.20689655172413793 星影のエール_A
0.13846153846153847 星影のエール_S
0.07964601769911504 涙空_A
0.27586206896551724 涙空_S
0.20689